In [15]:
# First we will import the necessary Library 

import os
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

# For Evalution we will use these library

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model


# For model building we will use these library

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM


# For PLotting we will use these library

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [16]:
maindf = pd.read_csv("BTC-USD.csv")
# If dataset had null values we can use this code to drop all the null values present in the dataset
maindf=maindf.dropna()
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

In [26]:
closedf = maindf[['Date','Close']]
close_stock = closedf.copy()
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (365, 2)


In [27]:
# deleting date column and normalizing using MinMax Scaler

del closedf['Date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(365, 1)
[[0.33191126]
 [0.30140159]
 [0.28955499]
 [0.26813744]
 [0.23356855]
 [0.23174999]
 [0.23542719]
 [0.29808979]
 [0.29532979]
 [0.31935223]
 [0.39242228]
 [0.39869596]
 [0.38937825]
 [0.34029395]
 [0.28060241]
 [0.2372274 ]
 [0.29680274]
 [0.32080605]
 [0.33867598]
 [0.36242053]
 [0.33485362]
 [0.44931059]
 [0.50993126]
 [0.4993531 ]
 [0.49485198]
 [0.46469067]
 [0.4479549 ]
 [0.45759293]
 [0.37391655]
 [0.36573667]
 [0.47915054]
 [0.54041473]
 [0.53778169]
 [0.52782569]
 [0.50640722]
 [0.50488475]
 [0.48234129]
 [0.4735133 ]
 [0.45905474]
 [0.50321555]
 [0.48121185]
 [0.49560739]
 [0.53810725]
 [0.49483115]
 [0.54737439]
 [0.54803756]
 [0.57790633]
 [0.57933172]
 [0.57230642]
 [0.56007413]
 [0.54305786]
 [0.50634696]
 [0.49807903]
 [0.34144121]
 [0.36079352]
 [0.38548201]
 [0.3764123 ]
 [0.38507732]
 [0.37615146]
 [0.38996164]
 [0.30001878]
 [0.28537707]
 [0.25687435]
 [0.30256601]
 [0.26894787]
 [0.2859154 ]
 [0.29110563]
 [0.28054974]
 [0.2713143 ]
 [0.28168612]
 [0.2699917

In [19]:
# we keep the training set as 70% and 30% testing set

training_size=int(len(closedf)*0.70)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (255, 1)
test_data:  (110, 1)


In [20]:
# convert an array of values into a dataset matrix

def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [21]:
time_step = 15
X_test, y_test = create_dataset(closedf, time_step)

In [22]:
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [23]:
model = load_model('btc_price_prediction.h5')

In [24]:
predict = model.predict(X_test)
predict_inv = scaler.inverse_transform(predict)
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

11/11 [==============================] - 0s 2ms/step


In [28]:
# shift train predictions for plotting

look_back=time_step
# trainPredictPlot = np.empty_like(closedf)
# trainPredictPlot[:, :] = np.nan
# trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
# print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[(look_back):len(closedf)-1, :] = predict_inv
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],
                                          plotdf['predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Test predicted data:  (365, 1)


In [31]:
import dash
from dash import dcc
from dash import html
import plotly.graph_objs as go
from dash.dependencies import Input, Output

In [32]:
# Run a Dash app

app = dash.Dash()
server = app.server

In [34]:
app.layout = html.Div([
    html.H1("Bitcoin Price Prediction", style={"textAlign": "center"}),
    html.Div([
        dcc.Dropdown(id='my-dropdown',
                        options=[{'label': 'BTC/USD', 'value': 'BTC/USD'}],
                        multi=True,value=['BTC/USD'],
                        style={"display": "block", "margin-left": "auto", "margin-right": "auto", "width": "60%"}             
        )
    ]),
    dcc.Graph(id="btc-graph")
])

@app.callback(Output("btc-graph", "figure"), [Input("my-dropdown", "value")])
def update_graph(selected_dropdown):
    dropdown = {"BTC/USD": "BTC-USD"}
    trace1 = []
    trace2 = []

    # Load the data
    df = pd.read_csv("BTC-USD.csv")
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df = df.dropna()
    closedf = maindf[['Date','Close']]
    close_stock = closedf.copy()
    del closedf['Date']
    scaler=MinMaxScaler(feature_range=(0,1))
    closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
    time_step = 15
    X, y = create_dataset(closedf, time_step)
    X = X.reshape(X.shape[0],X.shape[1] , 1)
    model = load_model('btc_price_prediction.h5')

    predict = model.predict(X)
    predict_inv = scaler.inverse_transform(predict)
    original_ytest = scaler.inverse_transform(y.reshape(-1,1)) 


    look_back=time_step
    # trainPredictPlot = np.empty_like(closedf)
    # trainPredictPlot[:, :] = np.nan
    # trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
    # print("Train predicted data: ", trainPredictPlot.shape)

    # shift test predictions for plotting
    testPredictPlot = np.empty_like(closedf)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[(look_back):len(closedf)-1, :] = predict_inv

    names = cycle(['Original close price','Predicted close price'])

    trace1.append(go.Scatter(x=df.Date, y=df.Close, mode='lines', opacity=0.7, name="Actual Price",
                                textposition='bottom center'))

    trace2.append(go.Scatter(x=df.Date, y=testPredictPlot.reshape(1,-1)[0].tolist(), mode='lines', opacity=0.6, name="Predicted Price",
                                textposition='bottom center')) 

    traces = [trace1, trace2]

    data = [val for sublist in traces for val in sublist]

    figure = {'data': data,
                'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056'],
                                    height=600,
                                    title=f"Actual and Predicted Prices for {selected_dropdown} Over Time",
                                    xaxis={"title":"Date",
                                            'rangeselector': {'buttons': list([{'count': 1, 'label': '1M',
                                                                                'step': 'month',
                                                                                'stepmode': 'backward'},
                                                                                {'count': 6, 'label': '6M',
                                                                                'step': 'month',
                                                                                'stepmode': 'backward'},
                                                                                {'step': 'all'}])},
                                                                                'rangeslider': {'visible': True}, 'type': 'date'},
                                    yaxis={"title":"Price (USD)"})}
    return figure

    # Predict the price


if __name__=='__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
11/11 [==============================] - 0s 3ms/step
